![An interactive LADAL notebook](https://slcladal.github.io/images/uq1.jpg)

***

Please copy this Jupyter notebook so that you are able to edit it.

Simply go to: File > Save a copy in Drive.

Once you have done that, you are good to go.

***

This tutorial is the interactive Jupyter notebook accompanying the [Language Technology and Data Analysis Laboratory (LADAL) tutorial on part-of-speech tagging and dependency parsing with R](https://ladal.edu.au/postag.html). 



***


**Preparation and session set up**

If you are using this notebook on Google Colab or your own computer and you have not already installed the R packages listed below, you need to install them. You can install them by running the code chunk below. 


In [ ]:
# install packages
install.packages("dplyr") 
install.packages("stringr")
install.packages("udpipe")


***



In [ ]:
# activate packages
library(dplyr)
library(stringr) 
library(udpipe) 


Once you have initiated the session by executing the code shown above, you are good to go.

***

## Using your own data

While the tutorial uses data from the LADAL website, you can also use your own data. You can see below what you need to do to upload and use your own data.

The code chunk below allows you to upload two files from your own computer. To be able to load your own data, you need to click on the folder symbol to the left of the screen:

![Colab Folder Symbol](https://slcladal.github.io/images/ColabFolder.png)

Then on the upload symbol.

![Colab Upload Symbol](https://slcladal.github.io/images/ColabUpload.png)

Next, upload the files you want to analyze and then the respective files names in the file argument of the scan function. When you then execute the code (like to code chunk below, you will upload your own data.


In [ ]:
mytext1 <- scan(file = "linguistics01.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
mytext2 <- scan(file = "linguistics02.txt",
            what = "char", 
            sep = "", 
            quote = "", 
            quiet = T, 
            skipNul = T) %>%
            paste0(collapse = " ")
# inspect
mytext1; mytext2


**Keep in mind though that you need to adapt the names of the texts in the code chunks below so that the code below work on your own texts!**

***

# POS-Tagging with UDPipe

UDPipe allows you to access numerous language models for 64 languages (for an overview of the supported languages and language models see [here](https://ladal.edu.au/postag.html#POS-Tagging_with_UDPipe)). 

To download any of these models, we can use the `udpipe_download_model` function. For example, to download the `english-ewt` model, we would use the call: `m_eng	<- udpipe::udpipe_download_model(language = "english-ewt")`. 

We start by loading  a text


In [ ]:
# load text
text <- readLines("https://slcladal.github.io/data/testcorpus/linguistics06.txt", skipNul = T) %>%
 str_squish() %>%
  .[1]
# inspect
text


Now that we have a text that we can work with, we will download a pre-trained language model.



In [ ]:
# download language model
m_eng	<- udpipe::udpipe_download_model(language = "english-ewt")


We now load language model.



In [ ]:
# load language model
m_eng <- udpipe_load_model(file = "/content/english-ewt-ud-2.5-191206.udpipe")


We can now use the model to annotate out text.



In [ ]:
# tokenise, tag, dependency parsing
text_anndf <- udpipe::udpipe_annotate(m_eng, x = text) %>%
  as.data.frame() %>%
  dplyr::select(-sentence)
# inspect
head(text_anndf, 10)


It can be useful to extract only the words and their pos-tags and convert them back into a text format (rather than a tabular format). 



In [ ]:
tagged_text <- paste(text_anndf$token, "/", text_anndf$xpos, collapse = " ", sep = "")
# inspect tagged text
tagged_text


# POS-Tagging non-English texts

We can apply the same method for annotating, e.g. adding pos-tags, to other languages. For this, we could train our own model, or, we can use one of the many pre-trained language models that `udpipe` provides.

Let us explore how to do this by using  example texts from different languages, here from German and Spanish (but we could also annotate texts from any of the wide variety of languages for which UDPipe provides pre-trained models.


We begin by loading a German and a Dutch text.


In [ ]:
# load texts
gertext <- readLines("https://slcladal.github.io/data/german.txt") 
duttext <- readLines("https://slcladal.github.io/data/dutch.txt") 
# inspect texts
gertext; duttext


Next, we install the pre-trained language models.



In [ ]:
# download language model
m_ger	<- udpipe::udpipe_download_model(language = "german-gsd")
m_dut	<- udpipe::udpipe_download_model(language = "dutch-alpino")


Or we load them from our machine (if we have downloaded and saved them before).



In [ ]:
# load language model from your computer after you have downloaded it once
m_ger	<- udpipe::udpipe_load_model(file = "/content/german-gsd-ud-2.5-191206.udpipe")
m_dut	<- udpipe::udpipe_load_model(file = "/content/dutch-alpino-ud-2.5-191206.udpipe")


Now, pos-tag the German text.



In [ ]:
ger_pos <- udpipe::udpipe_annotate(m_ger, x = gertext) %>%
  as.data.frame() %>%
  dplyr::summarise(postxt = paste(token, "/", xpos, collapse = " ", sep = "")) %>%
  dplyr::pull(unique(postxt))
# inspect
ger_pos


And finally, we also pos-tag the Dutch text.



In [ ]:
nl_pos <- udpipe::udpipe_annotate(m_dut, x = duttext) %>%
   as.data.frame() %>%
  dplyr::summarise(postxt = paste(token, "/", xpos, collapse = " ", sep = "")) %>%
  dplyr::pull(unique(postxt))
# inspect
nl_pos


# Dependency Parsing Using UDPipe

In addition to pos-tagging, we can also generate plots showing the syntactic dependencies of the different constituents of a sentence. For this, we generate an object that contains a sentence (in this case, the sentence *Linguistics is the scientific study of language*), and we then plot (or visualize) the dependencies using the `textplot_dependencyparser` function.  


In [ ]:
# parse text
sent <- udpipe::udpipe_annotate(m_eng, x = "Linguistics is the scientific study of language") %>%
  as.data.frame()
# inspect
head(sent)


Before we can generate the plot, we need to install additional packages.



In [ ]:
# install and activate necessary packages
install.packages("igraph")
install.packages("ggraph")
install.packages("textplot")


We now generate the plot.



In [ ]:
# load package
library(textplot)
# generate dependency plot
dplot <- textplot_dependencyparser(sent, size = 5) 
# show plot
dplot


That's it for this tutorial. We hope that you have enjoyed this tutorial and learned how to annotate texts using language models and perform pos-tagging and dependency parsing.



[Back to LADAL](https://ladal.edu.au/postag.html)
